In [1]:
import sys
sys.path.append("/anaconda/envs/azureml_py38/lib/python3.8/site-packages")

In [2]:
import psycopg2
import pandas as pd
import numpy as np

In [3]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'historical'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [4]:
conn, cur = get_db_cursor()


In [5]:
v1_samples = pd.read_csv('./outputFiles/spectra.csv',usecols=[0])

In [6]:
v1_samples

,Unnamed: 0
0,AGD207-408SA0706
1,AGD207-431SA0001
2,AGD207-431SA0002
3,AGD207-431SA0003
4,AGD207-431SA0004
...,...
87362,CW017SA7164
87363,CW017SA7167
87364,CW017SA7168
87365,CW104SA0052


In [7]:
v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)

/tmp/ipykernel_5014/2923516565.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  v2_samples = pd.read_sql('select sample_code from MandatoryMetadata', con = conn)


In [8]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
186311,CV062SA0752
186312,CH213SA0001
186313,CP433SA0007
186314,CP433SA0001


In [9]:
v2_samples = v2_samples.loc[~v2_samples['sample_code'].isin(v1_samples['Unnamed: 0'])]

In [10]:
v2_samples

,sample_code
0,ML-SS21-2
1,ML-SS21-2
2,ML-SS21-2
3,ML-SS21-2
4,ML-SS21-2
...,...
186311,CV062SA0752
186312,CH213SA0001
186313,CP433SA0007
186314,CP433SA0001


In [11]:
sample_codes = v2_samples['sample_code'].values

In [12]:
if(len(sample_codes) < 5000):
    count  = len(sample_codes)
    step=count
elif(len(sample_codes) < 70000):
    count = len(sample_codes)
    step=5000
else:
    count = 70000
    step=5000
start = 0


In [13]:
count

70000

In [14]:
start

0

In [15]:
step

5000

In [16]:
spectra = pd.DataFrame()

In [17]:
def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['averaged_spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [18]:
for i in np.arange(start, count, step):
        
    print("Fetching spectra from {}".format(start))
    samples = [i for i in sample_codes][start:start+step]
    print(len(samples))
    query = f"SELECT spectraldata.metadata_id, spectraldata.averaged_spectra, mandatorymetadata.sample_code FROM spectraldata INNER JOIN mandatorymetadata ON mandatorymetadata.metadata_id = spectraldata.metadata_id WHERE averaged=True AND sample_code IN {str(samples).replace('[','(').replace(']',')')}"

    _ = pd.read_sql(query, con=conn)
    print("This is the spectra",_)
    spectra = pd.concat([spectra, _], axis=0)
    start = start + step
    if (count-step) > 5000:
        step=5000
    else:
        step = count-step





Fetching spectra from 0
5000
This is the spectra       metadata_id                                   averaged_spectra  \
0          187056  {'522': [2.3645274701916428, 2.380267433090121...   
1          186833  {'522': [2.324679546571156, 2.3247525703442165...   
2          186966  {'522': [2.397271446121614, 2.4282403701836808...   
3          187043  {'522': [2.3943149118512363, 2.391047759744343...   
4          187025  {'522': [2.381931136382427, 2.383939558800457,...   
...           ...                                                ...   
6205       161700  {'522': [2.360054879422215, 2.302488578286381,...   
6206       161726  {'522': [2.400198791126645, 2.4553677734575245...   
6207       161730  {'522': [2.503719823794537, 2.5818505003199634...   
6208       161766  {'522': [2.242610077256419, 2.2293044933304005...   
6209       161777  {'522': [2.32940209441737, 2.2779935330976473,...   

          sample_code  
0           ML-SS21-1  
1           ML-SS21-1  
2           ML

/tmp/ipykernel_5014/1108552379.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_5014/1108552379.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_5014/1108552379.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  _ = pd.read_sql(query, con=conn)
/tmp/ipykernel_5014/1108552379.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 o

In [19]:
spectra

,metadata_id,averaged_spectra,sample_code
0,187056,"{'522': [2.3645274701916428, 2.380267433090121...",ML-SS21-1
1,186833,"{'522': [2.324679546571156, 2.3247525703442165...",ML-SS21-1
2,186966,"{'522': [2.397271446121614, 2.4282403701836808...",ML-SS21-1
3,187043,"{'522': [2.3943149118512363, 2.391047759744343...",ML-SS21-1
4,187025,"{'522': [2.381931136382427, 2.383939558800457,...",ML-SS21-1
...,...,...,...
5709,166338,"{'522': [2.431711745778313, 2.491956221424079,...",ML-SS21-3
5710,166368,"{'522': [2.3832986618691603, 2.418019421881964...",ML-SS21-3
5711,161316,"{'522': [2.388889688560504, 2.3500330274188115...",ML-SS21-1
5712,166389,"{'522': [2.477091493551886, 2.414741370768122,...",ML-SS21-3


In [39]:
df_ = spectra.copy(deep=True)

In [40]:
df_['spectra'] = df_['averaged_spectra'] + df_['non_averaged_spectra']

In [47]:
averaged = df_.loc[df_['non_averaged_spectra'].isna()]
not_averaged = df_.loc[df_['averaged_spectra'].isna()]
averaged['spectra'] = averaged['averaged_spectra']
not_averaged['spectra'] = not_averaged['non_averaged_spectra']
df = pd.concat([averaged, not_averaged])


/tmp/ipykernel_67100/1084805249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  averaged['spectra'] = averaged['averaged_spectra']
/tmp/ipykernel_67100/1084805249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_averaged['spectra'] = not_averaged['non_averaged_spectra']


In [48]:
df

,metadata_id,non_averaged_spectra,averaged_spectra,sample_code,spectra
0,179670,None,"{'521': [2.2655911367057704, 2.273310992571514...",ML-SS21-1,"{'521': [2.2655911367057704, 2.273310992571514..."
5,179712,None,"{'521': [2.2946008508409697, 2.27379168724669,...",ML-SS21-1,"{'521': [2.2946008508409697, 2.27379168724669,..."
6,179754,None,"{'521': [2.235812133945057, 2.3084154307617473...",ML-SS21-1,"{'521': [2.235812133945057, 2.3084154307617473..."
7,179814,None,"{'521': [2.3830898348113863, 2.376923259819746...",ML-SS21-1,"{'521': [2.3830898348113863, 2.376923259819746..."
8,179874,None,"{'522': [2.2885870694773565, 2.306158053862360...",ML-SS21-1,"{'522': [2.2885870694773565, 2.306158053862360..."
...,...,...,...,...,...
18782,174324,"{'522': [2.0846219245290767, 2.1293823581819, ...",None,CF177SA4851,"{'522': [2.0846219245290767, 2.1293823581819, ..."
18783,174325,"{'522': [2.2313015966713303, 2.22548869635572,...",None,CF177SA4847,"{'522': [2.2313015966713303, 2.22548869635572,..."
18784,174325,"{'522': [2.314154897949655, 2.283712153409843,...",None,CF177SA4847,"{'522': [2.314154897949655, 2.283712153409843,..."
18787,174326,"{'522': [2.1735385023639964, 2.113213803092161...",None,CF177SA4752,"{'522': [2.1735385023639964, 2.113213803092161..."


In [51]:
conn.close()
spectra = df[['sample_code', 'spectra']]


In [52]:
spectra = spectra.set_index('sample_code')


In [20]:
spectra = convertSpectra(spectra)


spectra.to_csv("./outputFiles/spectraldata.csv")

Converting spectra
Spectra converted


In [21]:
spectra

,522,524,526,528,530,532,534,536,538,540,...,3958,3960,3962,3964,3966,3968,3970,3972,3974,3976
0,2.364527,2.380267,2.415282,2.431268,2.441101,2.462991,2.473978,2.428879,2.422211,2.447353,...,1.170164,1.170703,1.170675,1.170313,1.170076,1.169900,1.169492,1.168838,1.168238,1.167781
1,2.324680,2.324753,2.373763,2.411027,2.402217,2.347667,2.317289,2.338107,2.375101,2.368011,...,1.149651,1.149604,1.149459,1.148917,1.148238,1.147781,1.147594,1.147623,1.147640,1.147664
2,2.397271,2.428240,2.399225,2.358932,2.332269,2.308703,2.294938,2.292284,2.298963,2.309691,...,1.160591,1.160070,1.159524,1.159550,1.159486,1.158533,1.157588,1.157537,1.157760,1.157822
3,2.394315,2.391048,2.357433,2.361828,2.380389,2.410625,2.426476,2.401460,2.370650,2.361235,...,1.152852,1.152934,1.152444,1.151799,1.151322,1.150696,1.150411,1.150530,1.150366,1.150150
4,2.381931,2.383940,2.376204,2.340311,2.348089,2.348603,2.345284,2.341323,2.328165,2.294485,...,1.159744,1.158565,1.157161,1.156499,1.156687,1.156656,1.155951,1.155213,1.154969,1.155184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5709,2.431712,2.491956,2.525230,2.497294,2.461680,2.457347,2.479673,2.508331,2.492839,2.441814,...,1.129957,1.129489,1.128628,1.128164,1.128154,1.128244,1.128160,1.127920,1.127710,1.127501
5710,2.383299,2.418019,2.405626,2.382745,2.379580,2.403409,2.391373,2.320085,2.273765,2.275257,...,1.149114,1.149265,1.149298,1.148614,1.147901,1.148023,1.148273,1.148000,1.147757,1.147768
5711,2.388890,2.350033,2.309996,2.327226,2.352492,2.330514,2.316581,2.350303,2.387681,2.412264,...,1.171154,1.170969,1.170589,1.170483,1.170664,1.170294,1.169510,1.169155,1.169263,1.169555
5712,2.477091,2.414741,2.403005,2.449703,2.481381,2.444921,2.382814,2.372414,2.406186,2.411119,...,1.161688,1.161186,1.160171,1.159406,1.159063,1.158751,1.158270,1.157872,1.157729,1.157586


In [22]:
import pandas as pd
spectra= pd.read_csv("outputFiles/spectraldata.csv",usecols=[0])
len(spectra)

123300